This is jupyter notebook for test training runs

In [1]:
import pandas as pd
import numpy as np
from FNN import FNN
from gradient_descent import *
from ActivFunctions import  *
from LossFunctions import *

In [2]:
path_data = "data\wdbc.data"#data from https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic
regGoal = "radius1"

In [3]:
epochs = 1000
learning_rate = 0.001
batch_size = 24
loss_derivative = MeanSquaredErrorDerivative

In [4]:
columnNames = ["ID number","Diagnosis","radius1","texture1","perimeter1","area1","smoothness1","compactness1","concavity1","concave points1","symmetry1","fractal dimension1","radius2","texture2","perimeter2","area2","smoothness2","compactness2","concavity2","concave points2","symmetry2","fractal dimension2","radius3","texture3","perimeter3","area3","smoothness3","compactness3","concavity3","concave points3","symmetry3","fractal dimension3"]
data_raw = pd.read_csv(path_data,names = columnNames)
data_NoId = data_raw.drop(columns="ID number")#no need for id

In [5]:
#
data = data_NoId.drop(columns="Diagnosis")#categorical variable is unneeded
data = data.iloc[:,0:10]#using only mean features
dataInput = data.drop(columns=regGoal).to_numpy()
dataTarget = data.loc[:,regGoal].to_numpy()
#
dataInput = dataInput.T
dataTarget = dataTarget[np.newaxis,:]

In [6]:
#declaring network
net = FNN([9,64, 32, 16, 8, 4, 2, 1],
          [relu,identity],
          method_ini="HeUni"
)

In [7]:
n_samples = dataTarget.size
indices = np.arange(0,n_samples)
indices_test = np.random.permutation(round(n_samples*0.2))
indices_train = list(set(indices) - set(indices_test))

In [8]:
#
dataInput_train = dataInput[:,indices_train]
dataTarget_train = dataTarget[:,indices_train]
#
dataInput_test = dataInput[:,indices_test]
dataTarget_test = dataTarget[:,indices_test]
#
meanDataInput_train = np.mean(dataInput_train, axis=1)
meanDataInput_train = meanDataInput_train[:,np.newaxis]
stdDataInput_train = np.std(dataInput_train, axis=1)
stdDataInput_train = stdDataInput_train[:,np.newaxis]
dataInput_train = (dataInput_train - meanDataInput_train) / stdDataInput_train
dataInput_test =  (dataInput_test - meanDataInput_train) / stdDataInput_train

In [9]:
net_trained = train_minibatch_sgd(net, dataInput_train, dataTarget_train, epochs, learning_rate, batch_size, loss_derivative, grad_clip = 2)

In [10]:
out = net_trained.forward(dataInput_test)
predict = out[-1]
error = MeanSquaredError(dataTarget_test,predict[-1])
print(error)

36.8612812514657


In [11]:
net_trained_adam = train_minibatch_adam(net,dataInput_train,dataTarget_train,epochs,learning_rate,batch_size,loss_derivative,beta1=0.9,beta2=0.999,epsilon=1e-8)

In [12]:
out = net_trained_adam.forward(dataInput_test)
predict = out[-1]
error = MeanSquaredError(dataTarget_test,predict[-1])
print(error)

10.438909196565278


Class

In [13]:
epochs = 1000
learning_rate = 0.001
batch_size = 24
loss_derivative = SoftmaxCrossEntropyDerivative

In [14]:
dataClass = data_NoId
dataInput= dataClass.iloc[:,1:].to_numpy()#all features without without diagn
dataTarget = dataClass.loc[:,"Diagnosis"].to_numpy()
#
dataInput = dataInput.T
dataTarget = dataTarget[np.newaxis,:]
dataTarget_one_hot = one_hot_encode(dataTarget,2)

In [15]:
n_samples = dataTarget.size
indices = np.arange(0,n_samples)
indices_test = np.random.permutation(round(n_samples*0.2))
indices_train = list(set(indices) - set(indices_test))

In [16]:
#
dataInput_train = dataInput[:,indices_train]
dataTarget_train = dataTarget_one_hot[:,indices_train]
#
dataInput_test = dataInput[:,indices_test]
dataTarget_test = dataTarget_one_hot[:,indices_test]
#
meanDataInput_train = np.mean(dataInput_train, axis=1)
meanDataInput_train = meanDataInput_train[:,np.newaxis]
stdDataInput_train = np.std(dataInput_train, axis=1)
stdDataInput_train = stdDataInput_train[:,np.newaxis]
dataInput_train = (dataInput_train - meanDataInput_train) / stdDataInput_train
dataInput_test =  (dataInput_test - meanDataInput_train) / stdDataInput_train

In [17]:
#declaring network
net = FNN([30,64, 32, 16, 8, 4, 2, 2],
          [relu,softmax],
          method_ini="HeUni"
)

In [18]:
net_trained = train_minibatch_sgd(net, dataInput_train, dataTarget_train, epochs, learning_rate, batch_size, loss_derivative, grad_clip = 2)

In [19]:
out = net_trained.forward(dataInput_test)
predict = out[-1]
error = MeanSquaredError(dataTarget_test,predict[-1])
print(error)

67.99942818356288


In [20]:
net_trained_adam = train_minibatch_adam(net,dataInput_train,dataTarget_train,epochs,learning_rate,batch_size,loss_derivative,beta1=0.9,beta2=0.999,epsilon=1e-8)

In [ ]:
out = net_trained_adam.forward(dataInput_test)
print(predict[-1].shape)
error = MeanSquaredError(dataTarget_test,predict[-1])
print(error)

(2, 114)
4.736686662688979
